In [1]:
!pip install mlflow

     |████████████████████████████████| 10.4 MB 10.1 MB/s eta 0:00:01███▉                       | 2.9 MB 10.1 MB/s eta 0:00:01      | 5.6 MB 10.1 MB/s eta 0:00:01█████████████▉        | 7.7 MB 10.1 MB/s eta 0:00:01████████████████████  | 9.8 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 2.4 MB/s  eta 0:00:01
     |████████████████████████████████| 40 kB 8.1 MB/s  eta 0:00:01
     |████████████████████████████████| 94 kB 5.6 MB/s  eta 0:00:01
     |████████████████████████████████| 451 kB 71.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 58.6 MB/s eta 0:00:01��████▋                       | 1.6 MB 58.6 MB/s eta 0:00:01██████████████████▎         | 4.2 MB 58.6 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 81.1 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 298 kB 71.2 MB/s eta 0:00:01

In [17]:
import mlflow
mlflow.set_tracking_uri("http://10.255.128.25:5000")

In [18]:
from  mlflow.tracking import MlflowClient
client = MlflowClient()
experiments = client.list_experiments()
experiments

[<Experiment: artifact_location='s3://mlflow/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlflow/1', experiment_id='1', lifecycle_stage='active', name='ishii_test', tags={}>]

In [19]:
experiments[1].experiment_id

'1'

In [22]:
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)

# Read the wine-quality csv file from the URL
csv_url =\
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
try:
    data = pd.read_csv(csv_url, sep=';')
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, check your internet connection. Error: %s", e)

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# ここのパラメータを書き換える
alpha = 0.5
l1_ratio = 0.5

with mlflow.start_run(experiment_id=experiments[1].experiment_id):
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.793164022927685
  MAE: 0.6271946374319586
  R2: 0.10862644997792636


In [9]:
!pip install boto3

     |████████████████████████████████| 128 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 6.2 MB 24.2 MB/s eta 0:00:01████████▎                       | 1.6 MB 24.2 MB/s eta 0:00:01��█▋          | 4.2 MB 24.2 MB/s eta 0:00:01
     |████████████████████████████████| 547 kB 51.0 MB/s eta 0:00:01
